# Pobieranie danych z Crossref
W tej części kodu korzystając z bazy Crossref wyszukujemy po nazwisku publikacje, które są autorstwa danego ekonomisty.

Najpierw ładujemy biblioteki i wczytujemy wcześniej zwebscrapowane dane

In [35]:
import requests
import json
import pandas as pd
import re

In [44]:
df_razem = pd.read_csv("Polish_all_economists_30_12.csv")
df_razem = df_razem.drop("Unnamed: 0", axis=1)
df_razem.head(7)

,Autor,ID,Strona_Ideas,Rank,W.Rank,Instytucja,Score,Imię,Nazwisko,Płeć,H_index,i10_index,Cytowania,Auto_cytowania,Artykuły,Papers,Książki,Rozdziały,Lata_działalności,CitEc_Link
0,Oded Stark,pst393,/f/pst393.html,1,[1],"Wydział Nauk Ekonomicznych, Uniwersytet Warsz...",1.27,Oded,Stark,M,32.0,68.0,6939.0,264 (3.67 %),217.0,276.0,4.0,15.0,47 years (1976 - 2023).,https://citec.repec.org/p/s/pst393.html
1,Rafal Weron,pwe42,/e/pwe42.html,2,[2],Katedra Badań Operacyjnych i Inteligencji Biz...,2.77,Rafal,Weron,M,32.0,73.0,3638.0,155 (4.09 %),62.0,151.0,3.0,3.0,28 years (1995 - 2023).,https://citec.repec.org/p/w/pwe42.html
2,Marcin Kolasa,pko253,/f/pko253.html,3,[4],"Katedra Ekonomii Ilościowej, Szkoła Główna Ha...",5.37,Marcin,Kolasa,M,20.0,27.0,1105.0,55 (4.74 %),44.0,89.0,NaN,1.0,19 years (2004 - 2023).,https://citec.repec.org/p/k/pko253.html
3,Mikolaj Czajkowski,pcz11,/e/pcz11.html,4,[5],"Wydział Nauk Ekonomicznych, Uniwersytet Warsz...",6.86,Mikolaj,Czajkowski,M,22.0,36.0,1071.0,104 (8.85 %),57.0,111.0,1.0,10.0,15 years (2008 - 2023).,https://citec.repec.org/p/c/pcz11.html
4,Michal Brzoza-Brzezina,pbr47,/e/pbr47.html,5,[7],"Katedra Ekonomii Ilościowej, Szkoła Główna Ha...",7.13,Michal,Brzoza-Brzezina,M,14.0,23.0,688.0,56 (7.53 %),29.0,78.0,NaN,NaN,21 years (2002 - 2023).,https://citec.repec.org/p/b/pbr47.html
5,Michal Wiktor Krawczyk,pkr96,/e/pkr96.html,6,[12],Group for Research in Applied Economics (GRAP...,7.18,Michal,Wiktor Krawczyk,M,10.0,10.0,717.0,23 (3.11 %),27.0,47.0,NaN,NaN,12 years (2008 - 2020).,https://citec.repec.org/p/k/pkr96.html
6,Jakub Growiec,pgr106,/e/pgr106.html,7,[11],"Katedra Ekonomii Ilościowej, Szkoła Główna Ha...",8.37,Jakub,Growiec,M,13.0,17.0,498.0,62 (11.07 %),36.0,69.0,2.0,11.0,18 years (2005 - 2023).,https://citec.repec.org/p/g/pgr106.html


Będę potrzebował dokładnego imienia i nazwiska, zatem w kolumnie "Nazwisko" usunę drugie imię (zostawię tylko drugie słowo)

In [79]:
for i in range (0,len(df_razem)):
    df_razem.loc[i,"Nazwisko"] = re.sub(r'^\w+\s', '',df_razem.loc[i,"Nazwisko"])
    
df_razem.head(8)

,Autor,ID,Strona_Ideas,Rank,W.Rank,Instytucja,Score,Imię,Nazwisko,Płeć,H_index,i10_index,Cytowania,Auto_cytowania,Artykuły,Papers,Książki,Rozdziały,Lata_działalności,CitEc_Link
0,Oded Stark,pst393,/f/pst393.html,1,[1],"Wydział Nauk Ekonomicznych, Uniwersytet Warsz...",1.27,Oded,Stark,M,32.0,68.0,6939.0,264 (3.67 %),217.0,276.0,4.0,15.0,47 years (1976 - 2023).,https://citec.repec.org/p/s/pst393.html
1,Rafal Weron,pwe42,/e/pwe42.html,2,[2],Katedra Badań Operacyjnych i Inteligencji Biz...,2.77,Rafal,Weron,M,32.0,73.0,3638.0,155 (4.09 %),62.0,151.0,3.0,3.0,28 years (1995 - 2023).,https://citec.repec.org/p/w/pwe42.html
2,Marcin Kolasa,pko253,/f/pko253.html,3,[4],"Katedra Ekonomii Ilościowej, Szkoła Główna Ha...",5.37,Marcin,Kolasa,M,20.0,27.0,1105.0,55 (4.74 %),44.0,89.0,NaN,1.0,19 years (2004 - 2023).,https://citec.repec.org/p/k/pko253.html
3,Mikolaj Czajkowski,pcz11,/e/pcz11.html,4,[5],"Wydział Nauk Ekonomicznych, Uniwersytet Warsz...",6.86,Mikolaj,Czajkowski,M,22.0,36.0,1071.0,104 (8.85 %),57.0,111.0,1.0,10.0,15 years (2008 - 2023).,https://citec.repec.org/p/c/pcz11.html
4,Michal Brzoza-Brzezina,pbr47,/e/pbr47.html,5,[7],"Katedra Ekonomii Ilościowej, Szkoła Główna Ha...",7.13,Michal,Brzoza-Brzezina,M,14.0,23.0,688.0,56 (7.53 %),29.0,78.0,NaN,NaN,21 years (2002 - 2023).,https://citec.repec.org/p/b/pbr47.html
5,Michal Wiktor Krawczyk,pkr96,/e/pkr96.html,6,[12],Group for Research in Applied Economics (GRAP...,7.18,Michal,Krawczyk,M,10.0,10.0,717.0,23 (3.11 %),27.0,47.0,NaN,NaN,12 years (2008 - 2020).,https://citec.repec.org/p/k/pkr96.html
6,Jakub Growiec,pgr106,/e/pgr106.html,7,[11],"Katedra Ekonomii Ilościowej, Szkoła Główna Ha...",8.37,Jakub,Growiec,M,13.0,17.0,498.0,62 (11.07 %),36.0,69.0,2.0,11.0,18 years (2005 - 2023).,https://citec.repec.org/p/g/pgr106.html
7,Michal Rubaszek,pru76,/e/pru76.html,8,[10],"Instytut Ekonometrii, Szkoła Główna Handlowa ...",10.10,Michal,Rubaszek,M,11.0,17.0,545.0,42 (7.16 %),50.0,43.0,NaN,2.0,21 years (2002 - 2023).,https://citec.repec.org/p/r/pru76.html


Mając wyczyszczone imiona i nazwiska mogę przejść do pobierania danych z API.

Zaczynam od uruchomienia klienta API

In [80]:
url = 'https://api.crossref.org/works?'
headers = {'User-Agent': 'ScientificResearch/1.0 (mailto:t.metrak@student.uw.edu.pl) '}

Następnie buduję słownik, gdzie dla każdego ekonomisty zapiszę efekty wyszukiwania. Potem będę pracował na tych efektacg wyszukiwania.

In [97]:
CrossRef = {}
for i in range (0, len(df_razem)):
    query = f'query={df_razem.loc[i, "Imię"]}+{df_razem.loc[i, "Nazwisko"]}'
    r = requests.get(url+query, headers=headers)
    pubs = json.loads(r.text)
    CrossRef[f'id_{df_razem.loc[i, "ID"]}'] = {"pubs": pubs}
    
print(CrossRef)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [100]:
import pickle

# save dictionary to person_data.pkl file
with open('CrossRef.pkl', 'wb') as fp:
    pickle.dump(CrossRef, fp)

In [106]:
CrossRef["id_pst393"]["pubs"]["message"]["items"][1]["author"]

[{'given': 'Oded', 'family': 'Stark', 'sequence': 'first', 'affiliation': []}]

In [95]:
pubs["message"]["items"][12]["publisher"]

'Elsevier BV'

In [73]:
pubs["message"]["items"][1]["author"]

[{'given': 'Michal',
  'family': 'Brzoza-Brzezina',
  'sequence': 'first',
  'affiliation': []},
 {'given': 'Paweł',
  'family': 'Galiński',
  'sequence': 'additional',
  'affiliation': []},
 {'given': 'Krzysztof',
  'family': 'Makarski',
  'sequence': 'additional',
  'affiliation': []}]

In [99]:
len(CrossRef)

222